In [6]:
%load_ext autoreload
#To update the imported files

In [7]:
%autoreload
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import mean_absolute_error
from utilities import *
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [8]:
#Loading the y-paramter from file, might be changed to y_a etc.
train_a = pd.read_parquet('../Data_and_task/A/train_targets.parquet') #y verdi for estimert og observert 
train_b = pd.read_parquet('../Data_and_task/B/train_targets.parquet')
train_c = pd.read_parquet('../Data_and_task/C/train_targets.parquet')

In [9]:
#Loading estimated/forecasted training_weather from file
X_train_estimated_a = pd.read_parquet('../Data_and_task/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../Data_and_task/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../Data_and_task/C/X_train_estimated.parquet')

In [10]:
#Loading observed weather from file
X_train_observed_a = pd.read_parquet('../Data_and_task/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('../Data_and_task/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('../Data_and_task/C/X_train_observed.parquet')

In [12]:
#Loading estimated/forecasted test_weather from file
X_test_estimated_a = pd.read_parquet('../Data_and_task/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('../Data_and_task/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('../Data_and_task/C/X_test_estimated.parquet')

X_train_estimated_c.head()

,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2022-10-27 07:00:18,2022-10-28 22:00:00,8.3,1.231,1427.400024,0.0,0.0,1215.800049,1.0,281.200012,0.0,0.0,0.0,0.0,100.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1001.099976,0.0,0.0,986.000000,992.000000,0.0,0.0,93.900002,998.099976,NaN,0.0,0.0,-0.0,0.4,340.785004,-38.502998,0.0,285.200012,100.0,34387.398438,1.2,-0.3,1.1,-0.0
1,2022-10-27 07:00:18,2022-10-28 22:15:00,8.3,1.231,2089.000000,0.0,0.0,1425.900024,1.0,281.200012,0.0,0.0,0.0,0.0,100.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000.700012,0.0,0.0,985.700012,991.700012,0.0,0.0,93.599998,997.700012,NaN,0.0,0.0,-0.0,0.5,345.440002,-38.992001,0.0,285.100006,100.0,27418.500000,1.2,-0.2,1.2,-0.0
2,2022-10-27 07:00:18,2022-10-28 22:30:00,8.3,1.231,2750.500000,0.0,0.0,1636.099976,1.0,281.100006,0.0,0.0,0.0,0.0,100.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000.400024,0.0,0.0,985.400024,991.400024,0.0,0.1,93.300003,997.400024,NaN,0.0,0.0,-0.0,0.5,350.144012,-39.348000,0.0,285.100006,100.0,20508.800781,1.2,-0.1,1.2,-0.0
3,2022-10-27 07:00:18,2022-10-28 22:45:00,8.2,1.230,3412.100098,0.0,0.0,1846.199951,1.0,281.100006,0.0,0.0,0.0,0.0,100.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000.099976,0.0,0.0,985.099976,991.099976,0.0,0.1,93.000000,997.099976,NaN,0.0,0.0,-0.0,0.5,354.880005,-39.567001,0.0,285.000000,100.0,13657.799805,1.2,-0.0,1.2,-0.0
4,2022-10-27 07:00:18,2022-10-28 23:00:00,8.2,1.230,4073.699951,0.0,0.0,2056.399902,1.0,281.100006,0.0,0.0,0.0,0.0,100.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,999.700012,0.0,0.0,984.700012,990.700012,0.0,0.1,92.699997,996.799988,NaN,0.0,0.0,-0.0,0.6,359.634003,-39.647999,0.0,285.000000,100.0,6865.000000,1.2,0.1,1.2,-0.0


In [8]:
#Cleaning data such that only wanted features are included
selected_features = [
    "date_forecast", "clear_sky_rad:W", "diffuse_rad:W", "direct_rad:W",
    "absolute_humidity_2m:gm3", "effective_cloud_cover:p",
    "is_day:idx", "is_in_shadow:idx", "fresh_snow_12h:cm", "fresh_snow_1h:cm",
    "fresh_snow_24h:cm", "fresh_snow_3h:cm", "fresh_snow_6h:cm",
    "snow_depth:cm", "snow_melt_10min:mm", "sun_azimuth:d", 
    "sun_elevation:d", "wind_speed_10m:ms"
]
#Defining y_fetures, will always be the same
y_features = ["time", "pv_measurement"]

X_train_observed_a_clean = X_train_observed_a[selected_features].copy()
X_train_estimated_a_clean = X_train_estimated_a[selected_features].copy()

X_test_estimated_a_clean = X_test_estimated_a[selected_features][::4].drop(columns = ["date_forecast"]).copy()
X_test_estimated_b_clean = X_test_estimated_b[selected_features][::4].drop(columns = ["date_forecast"]).copy()
X_test_estimated_c_clean = X_test_estimated_c[selected_features][::4].drop(columns = ["date_forecast"]).copy()

In [9]:
#Training a Linear regression model on X_observed_a and testing it on X_estimated_a and evaluating it on MAE, PURELY for testing!
#See below for how its done when submitting
X_train_observed_a_resized, y_train_observed_a = resize_trainingdata(X_train_observed_a_clean, train_a, "date_forecast", y_features)
X_train_estimated_a_resized, y_train_estimated_a = resize_trainingdata(X_train_estimated_a_clean, train_a, "date_forecast", y_features)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_observed_a_resized)
X_test_scaled = scaler.fit_transform(X_train_estimated_a_resized)

#Training the model
reg = LinearRegression()
reg.fit(X_train_scaled, y_train_observed_a["pv_measurement"])

# Make predictions
y_pred = reg.predict(X_test_scaled)

#Evaluate mean asbolute error
mae = mean_absolute_error(y_train_estimated_a["pv_measurement"], y_pred)
print("Mean Absolute Error:", mae)



Mean Absolute Error: 382.9974707367402


In [1]:
df_corr = df[['Location', 'Time', 'Latitude', 'Longitude', 'Altitude',
              'YRMODAHRMI', 'Month', 'Hour', 'Season', 'Humidity', 'AmbientTemp',
              'Wind.Speed', 'Visibility', 'Pressure', 'Cloud.Ceiling', 'PolyPwr']].corr()

mask = np.triu(np.ones_like(df_corr, dtype=bool))

f, ax = plt.subplots(figsize=(11, 9))

sns.heatmap(df_corr, mask=mask, cmap='jet', vmax=.3, center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .8});

plt.title('Correlation analysis for all sites');

NameError: name 'df' is not defined

In [10]:
#Training model, running infernece on testing data and saving it in a csv file ready for submission
X_train_observed_a_resized, y_train_observed_a = resize_trainingdata(X_train_observed_a_clean, train_a, "date_forecast", y_features)
X_train_estimated_a_resized, y_train_estimated_a = resize_trainingdata(X_train_estimated_a_clean, train_a, "date_forecast", y_features)

#Merging observed and estimated data to one big df
X_train_a = pd.concat([X_train_observed_a_resized, X_train_estimated_a_resized], ignore_index=True)
y_train_a = pd.concat([y_train_observed_a, y_train_estimated_a], ignore_index=True)
X_test = pd.concat([X_test_estimated_a_clean, X_test_estimated_b_clean, X_test_estimated_c_clean], ignore_index=True)

#Scaling the data for more fair comparions and faster convergence, ChatGPT
scaler = StandardScaler()
X_train_a_scaled = scaler.fit_transform(X_train_a)
X_test_a_scaled=scaler.fit_transform(X_test)

#Training the models
#Model A:
reg_a = LinearRegression()
reg_a.fit(X_train_a, y_train_a["pv_measurement"])

# Make predictions for A,B,C on the model trained on only A. Problems with the B and C datasets which need to be explored
y_pred = reg_a.predict(X_test_a_scaled)

#Saving y_pred in a proper csv file
filename = "CSV/second_submission.csv"
submission(filename, y_pred)

C:\Users\Gustav\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
